# Свёрточные нейросети — классификация изображений (TensorFlow)
### 1. Инициализация

In [ ]:
# Импорт необходимых библиотек и модулей
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, Input
from tensorflow.keras.models import Sequential

# Проверка версии TensorFlow
tf.__version__

### 2. Загрузка датасета
Используем датасет [CIFAR‑10](https://www.cs.toronto.edu/~kriz/cifar.html) — классический бенчмарк для задач компьютерного зрения. В нём 60 000 цветных изображений 32×32 в 10 классах (по 6 000 изображений на класс). Датасет широко применяется для обучения и проверки алгоритмов классификации изображений.

Классы: самолёт, автомобиль, птица, кошка, олень, собака, лягушка, лошадь, корабль и грузовик.

In [ ]:
# Загрузка датасета из библиотеки Keras
from keras.datasets import cifar10 
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Проверка формы загруженных данных
print("X Train: {} \nX Test: {} \ny Train: {} \ny test: {}".format(X_train.shape, X_test.shape, y_train.shape, y_test.shape))


Как видно выше, доступно 50 000 обучающих и 10 000 тестовых изображений, каждое размера 32×32 пикселя с тремя цветовыми каналами (RGB). Метки классов присутствуют для обучающей и тестовой выборок и заданы в виде одномерных столбцов.

### 3. Предобработка изображений
Так как модель CNN решает многоклассовую задачу, целевые метки должны быть совместимы с выходом сети. Поэтому преобразуем метки классов в one‑hot представление, удобное для обучения нейросети.

In [ ]:
# Преобразование целевых меток к one-hot кодированию
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

Выполним аугментацию и преобразования изображений, чтобы уменьшить переобучение и повысить обобщающую способность модели.

Трансформации для обучающих изображений:
- Масштабирование пикселей к диапазону [0, 1];
- Случайный сдвиг по сдвигу (shear);
- Случайный зум;
- Случайное горизонтальное отражение.

Тестовые изображения не аугментируются — только нормируются, чтобы избежать утечки данных и корректно оценивать качество.

In [ ]:
# Инициализация генератора изображений для обучения
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

# Генерация батчей аугментированных обучающих данных
train_generator = train_datagen.flow(
    X_train, y_train,
    batch_size = 32
)

# Инициализация генератора изображений для теста (только масштабирование)
test_datagen = ImageDataGenerator(
    rescale = 1./255
)

# Генерация батчей тестовых данных
test_generator = test_datagen.flow(
    X_test, y_test,
    batch_size = 32
)

### 4. Построение модели CNN

In [ ]:
#### Инициализация модели CNN
# Инициализация последовательной модели
cnn = Sequential()

#### Входной слой
# Добавляем входной слой
cnn.add(Input(shape=(32, 32, 3)))

#### Свёрточные слои
# Используем 2 свёрточных слоя с увеличением числа фильтров, чтобы модель улавливала более сложные паттерны.
# За свёрточными слоями следуют слои максимального пуллинга, уменьшающие пространственные размеры карт признаков и частично
# снижающие переобучение за счёт устойчивости к сдвигам.

# Добавляем первый свёрточный слой
cnn.add(Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation="relu"))
# Добавляем первый слой пуллинга
cnn.add(MaxPool2D(pool_size=(2,2), strides=2))
# Добавляем второй свёрточный слой
cnn.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation="relu"))
# Добавляем второй слой пуллинга
cnn.add(MaxPool2D(pool_size=(2,2), strides=2))

#### Слой «выпрямления» (Flatten)
# Этот слой преобразует выход свёрточных слоёв в одномерный вектор для передачи в полносвязные слои.
cnn.add(Flatten())

#### Полносвязные слои
# Первый полносвязный слой обучается высокоуровневым признакам из «выпрямленного» входа,
# второй помогает дополнительно уточнить представления.
cnn.add(Dense(units=128, activation="relu"))
cnn.add(Dense(units=64, activation="relu"))

#### Выходной слой
# Финальный слой на 10 нейронов (по числу классов) с softmax для вероятностей классов.
cnn.add(Dense(units=10, activation="softmax"))

### 5. Обучение модели CNN

In [ ]:
#### Компиляция модели CNN
# Компилируем последовательную модель
cnn.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=[
        tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

#### Обучение модели CNN
# Обучаем модель на сгенерированных аугментированных данных
history = cnn.fit(
    train_generator,
    epochs=25,
    validation_data=test_generator
)

### 6. Анализ качества

In [ ]:
# Графики точности на обучении и валидации
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

Наблюдения по результатам:
- Точность на обучении последовательно растёт — модель осваивает задачу.
- Валидационная точность поначалу увеличивается, затем начинает колебаться — возможный признак надвигающегося переобучения.
- После пика (примерно около 9‑й эпохи) разрыв между обучением и валидацией расширяется.
- Чтобы бороться с переобучением, имеет смысл добавить регуляризацию (например, dropout, weight decay), раннюю остановку и/или усилить аугментации.

**Итог.** Модель демонстрирует уверенное начальное обучение, но позднее появляются симптомы переобучения. Для лучшей обобщающей способности полезно усилить регуляризацию и аккуратно настроить гиперпараметры (включая раннюю остановку).